In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.io
from pathlib import Path
import matplotlib.pyplot as plt
from sbi_ice.simulators import Layer_Tracing_Sim as lts
import pandas as pd
from tueplots import figsizes
from sbi_ice.utils import plotting_utils,misc
data_dir,output_dir = misc.get_data_output_dirs()
color_opts = plotting_utils.setup_plots()


# Lagrangian Mass Balance Plot

Here we create a figure comparing the features of the real data and simulated outputs, which we use to motivate future exploration of a Lagrangian paramtereization of mass balance.

In [ ]:
print(Path(data_dir,"Ekstrom","IRH_data","Ekstroem_flowline_GPR_IRH_data.mat"))
mat = scipy.io.loadmat(Path(data_dir,"Ekstrom","IRH_data","Ekstroem_flowline_GPR_IRH_data.mat"))
shelf_mask = scipy.io.loadmat(Path(data_dir,"Ekstrom","IRH_data","LayerData_flowline_mask.mat"))['save_in'].astype(bool).flatten()


## Load real layer data

In [ ]:
distance = mat["distance"].flatten()
gl_pos = distance[shelf_mask][0]

xmin,xmax = 100,125
xmask = (distance>xmin) & (distance<xmax)

fig,ax = plt.subplots(1,1,figsize=(5,5))
for i in range(0,mat["elev_layers"].shape[0]):
    ax.plot(distance[xmask],mat["elev_layers"][i,xmask],color = color_opts["colors"]["observation"])
base = mat["elev_sur_bedmachine"].flatten() - mat["thickness_bedmachine"].flatten()

#Also plot surface elevation
ax.plot(distance[xmask],mat["elev_sur_bedmachine"].flatten()[xmask],color="k",linewidth=2)

## Create illustrative exmaple domain for simulator and simulate layers

In [ ]:
data_file = Path(data_dir,"Toy","setup_files","flowtube_short.csv")

df = pd.read_csv(data_file)
xmb = df["x_coord"].to_numpy()
tmb = df["tmb"].to_numpy() 
Lx = xmb.max() - xmb.min()

nx_iso             = 200
ny_iso             = 1
dt                 = 0.5 

In [ ]:
geom = lts.Geom(nx_iso=nx_iso,ny_iso=ny_iso)

smb = 0.3 + 1.0*np.exp(-((xmb-0.1*Lx)/(0.05*Lx))**2)
bmb = 0.5*np.ones_like(xmb)
smb_regrid,bmb_regrid = lts.init_geom_from_fname(geom,data_file,smb=smb,bmb=bmb)

In [ ]:
dt=0.5
n_surface = 1
n_base = 10
time_phase = 500

sched = lts.Scheduele(time_phase,dt,n_surface,n_base)

In [ ]:
geom.initialize_layers(sched,10)
lts.sim(geom,smb_regrid,bmb_regrid,sched)

In [ ]:
plot_indices = [-50,-100,-150,-200]
fig,axs = plotting_utils.plot_isochrones_1d(geom.x,geom.bs.flatten(),geom.ss.flatten(),geom.dsum_iso[:,0,plot_indices],geom.age_iso[plot_indices],bmb_regrid,smb_regrid,real_layers=None,trackers=geom.extract_active_trackers())


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,5))
#Plot the plot_indices of the synthetic model
for idx in plot_indices:
    ax.plot(geom.x/1e3,geom.dsum_iso[:,0,idx]+geom.bs.flatten(),label=str(idx),color=color_opts["colors"]["prior"])

ax.plot(geom.x/1e3,geom.ss.flatten(),color="k")

# Plot comparison

In [ ]:
plt.rcParams.update(figsizes.icml2022_full(height_to_width_ratio=0.8))
plot_indices = [-100,-200,-300,-400]

fig, axs = plt.subplot_mosaic([["a", "b"], ["a", "c"]],
                              layout="constrained",
                              height_ratios = [0.3,0.7])


ax1 = axs["a"]
ax2 = axs["b"]
ax3 = axs["c"]
geom_mask = geom.x<25e3

ax1.plot(distance[xmask],mat["elev_sur_bedmachine"].flatten()[xmask],color="k",linewidth=2)
for i in range(0,mat["elev_layers"].shape[0]):
    ax1.plot(distance[xmask],mat["elev_layers"][i,xmask],color = color_opts["colors"]["observation"])
ax1.fill_between(distance[xmask],-100.0*np.ones_like(distance[xmask]),mat["elev_sur_bedmachine"].flatten()[xmask],color="k",alpha=0.075,linewidth=0.0)
ax1.spines["bottom"].set_bounds(distance[xmask][0]-0.001,distance[xmask][-1])
ax1.set_xlabel("Distance from GL [km]")
ax1.set_ylabel("Elevation [m.a.s.l.]")
ax1.annotate(r"$v_{x}$",xy=(115,41),xytext=(110,40),arrowprops=dict(arrowstyle="->"))
ax1.vlines(116.9,-90,(mat["elev_sur_bedmachine"].flatten()[xmask])[np.argmin(np.abs(distance[xmask]-116.9))],linestyle="--",color="k")
ax1.set_ylim(-90,45)


ax2.plot(geom.x[geom_mask]/1e3, smb_regrid.flatten()[geom_mask],color=color_opts["colors"]["prior"])
ax2.set_ylabel("$\dot{a}$ [m/a]")

ax3.plot(geom.x[geom_mask]/1e3, geom.ss.flatten()[geom_mask],color="k",linewidth=2)
for idx in plot_indices:
    ax3.plot(geom.x[geom_mask]/1e3,geom.dsum_iso[geom_mask,0,idx]+geom.bs.flatten()[geom_mask],label=str(idx),color=color_opts["colors"]["prior"])
ax3.fill_between(geom.x[geom_mask]/1e3,-90.0*np.ones_like(geom.x[geom_mask]),geom.ss.flatten()[geom_mask],color="k",alpha=0.075,linewidth=0.0)
ax3.spines["bottom"].set_bounds(geom.x[0]/1e3-0.001,geom.x[geom_mask][-1]/1e3)
ax3.set_xlabel("Distance [km]")
ax3.set_ylabel("Elevation [m.a.s.l.]")
ax3.annotate(r"$v_{x}$",xy=(20,26),xytext=(15,25),arrowprops=dict(arrowstyle="->"))

ax3.set_ylim(-60,30)

plt.setp(ax2.get_xticklabels(), visible=False)
ax2.tick_params(
    axis="x",          # changes apply to the x-axis
    which="both",      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
)
ax2.spines[["bottom","top","right"]].set_visible(False)

import matplotlib.transforms as mtransforms


for label, ax in axs.items():
    # label physical distance to the left and up:
    trans = mtransforms.ScaledTranslation(6/72, 0/72, fig.dpi_scale_trans)
    ax.text(0.0, 1.0, label, transform=ax.transAxes + trans,
            fontsize="medium", va="bottom", fontfamily="sans-serif")

# fig_name = Path(output_dir,"paper_figures","Lagrangian_MB","Lagrangian_MB.pdf")
# fig_name.parent.mkdir(parents=True,exist_ok=True)
# fig.savefig(fig_name)

